In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import random
import os

Δημιουργούμε τον φάκελο του current simulation. Αν υπάρχουν φάκελοι με αρίθμηση δημιουργούμε εναν καινούριο με index μεγαλύτερο κατά 1.

In [2]:
from pathlib import Path

# Get the directory of the script
parent_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()

# Initialize a list to store the indices
existing_indices = []

# Iterate through the folders in the directory
for folder in parent_dir.iterdir():
    if folder.is_dir() and folder.name.startswith("Sim_Data_"):  # Check if it's a directory and follows the pattern
        try:
            index = int(folder.name[len("Sim_Data_"):])  # Extract the number part
            existing_indices.append(index)
        except ValueError:
            pass  # Ignore folders that don't have a valid integer index

# Determine the next index
new_index = (max(existing_indices) + 1) if existing_indices else 0

# Create the new folder
new_folder_path = parent_dir / f"Sim_Data_{new_index}"
new_folder_path.mkdir(exist_ok=True)

print(f"Created folder: {new_folder_path}")



Created folder: c:\Users\jimka\OneDrive\Documents\GitHub\NNs-on-RFID-Localization\Simulation\Sim_Data_7


Κλάση Ρομπότ για την γέννηση των ρομπότ. Κάθε προσομοίωση έχει 1 ρομπότ συνήθως, με 1-4 κεραίες.

In [3]:
class Robot:
    def __init__(self,pos:np.ndarray,antennas:list):
        self.antennas = antennas
        self.position = pos
        self.num_of_antennas = len(antennas)
    def update_pos(self,pos):
        self.position = pos

Κλάση Tag για τη γέννηση των tags. Κάθε προσομοίωση έχει 1 tag.

In [5]:
class Tag:
    def __init__(self,pos:np.ndarray):
        self.position = pos

Κλάση Antenna για τις κεραίες των ρομπότ.

In [4]:
class Antenna:
    def __init__(self,z:float,G_dBm:float,theta_vector:np.ndarray,phi_vector:np.ndarray):
        self.z = z
        self.G_dBm = G_dBm
        self.theta_vector = theta_vector
        self.phi_vector = phi_vector
        self.phase_bias = random.uniform(0,2*np.pi)
    
    def get_gain(self,theta_rad:float,phi_rad:float):
        theta_deg: int = round(theta_rad*180/np.pi)%360 #we don't really need the modulo here, i think but just to be safe
        phi_deg: int = round(phi_rad * 180/np.pi)%360
        return self.theta_vector[theta_deg] + self.phi_vector[phi_deg] + self.G_dBm


Ξεκινάμε το simulation 

In [6]:
#We assume the RFID tag frequency is 866MHz
freq = 866e6
#Speed of light
c = 299792458.0

lambda_signal = c/freq


Pt_dBm = 30

G_R_dBm = 1

#antenna initialization
pattern = pd.read_excel("pattern.xlsx")
gain_theta = pattern["Gain_t(dB)"].to_numpy()
gain_phi = pattern["Gain_p(dB)"].to_numpy()
Antenna1 = Antenna(0,4,gain_theta,gain_phi)
Antenna2 = Antenna(0.4,4,gain_theta,gain_phi)
Antenna3 = Antenna(0.6,4,gain_theta,gain_phi)

#Points of the movement (for the moment it is straight)
p1 = np.array([0,0,0])
p2 = np.array([4,0,0])
#Initialization of robot with antennas
Robot_inst = Robot(p1,[Antenna1])
#p3 serves as a visual guide to know where the movement starts
kappa = 0.1
p3 = (1-kappa)*p1+kappa*p2


data_list = []  #list containing all the saved data
#tag position params
x_tag = 2 
z_tag = 0
y_steps = np.linspace(0.1,3,2000) #the last number is the number of samples for each epoch
for antenna_num,antenna in enumerate(Robot_inst.antennas):
    #do this for all antennas
    for y_tag in y_steps:
        #Tag initialization
        tagp = np.array([x_tag,y_tag,z_tag])
        tag1 = Tag(tagp)
        
        phases = []
        robot_pos = []
        rho = 0   #rho is the position of the x axis
        mu,sigma = 0.0038,0.001834  #mu and sigma for the normal distribution of the sampling based on real data. This can change depending on the data.
        while(rho<4):
        #for j in range(0,1000):
            rho = rho + np.random.normal(mu,sigma)
            #there will be a problem with z if p1 and p2 do not have z = 0
            new_pos = p1 + rho*(p2-p1)/np.linalg.norm(p2-p1)
            #new_pos = p1 + j*(p2-p1)/np.linalg.norm(p2-p1)
            robot_pos.append(new_pos)
            Robot_inst.update_pos(new_pos)
            r = np.linalg.norm(tag1.position-Robot_inst.position)
            #Find the phi,theta of the antenna pattern and move it from -pi,pi to 0,2pi
            if(tag1.position[0] == 0):
                theta_tag = np.pi/2
            else:
                theta_tag = np.atan(tag1.position[1]/tag1.position[0])
            theta_tag = (theta_tag + 2*np.pi)%np.pi
            if(tag1.position[2] == antenna.z):
                phi_tag = 0
            else:
                phi_tag = np.atan(np.sqrt(pow(tag1.position[0],2)+pow(tag1.position[1],2))/(tag1.position[2]-antenna.z))
            phi_tag = (phi_tag + 2*np.pi)%np.pi
            #we calculate the Friis equation
            Pr = antenna.get_gain(theta_tag,phi_tag) + G_R_dBm + Pt_dBm + 20*np.log10(lambda_signal/(4*np.pi*r))
            phases.append( ((2*np.pi)/lambda_signal)*2*r + antenna.phase_bias + np.random.normal(0,0.09)) #mu = 0 and sigma = 5deg ~= 0.09 rad
        data_list.append({
            "tag_pos": np.array([x_tag,y_tag,z_tag]),
            "Antenna": antenna_num,
            "robot_pos" : np.array(robot_pos),
            "Phases": np.array(phases)-phases[0]
        })
    
    ##VISUALIZATION

    # fig = plt.figure()
    # plt.scatter(np.array([p1[0],p3[0],p2[0],tagp[0]]),np.array([p1[1],p3[1],p2[1],tagp[1]]))
    # plt.show()
    
    # fig = plt.figure()
    # x = np.arange(0,len(phases))
    # plt.scatter(x,np.array(phases)-phases[0])
    # plt.show()

    # fig = plt.figure()
    # x = np.arange(0,len(phases))
    # plt.scatter(x,np.array(phases)%(2*np.pi),linewidths=0.2)
    # plt.show()

    # fig = plt.figure()
    # plt.plot(gain_theta)
    # plt.plot(gain_phi)
    # plt.show()

# Convert the list to a DataFrame
Data = pd.DataFrame(data_list)

# Save to a pickle file
Data.to_pickle(f"{new_folder_path}/Sim_Data_Noisy.pkl")

Data.head()



,tag_pos,Antenna,robot_pos,Phases
0,"[2.0, 0.1, 0.0]",0,"[[0.0016592182891534504, 0.0, 0.0], [0.0061772...","[0.0, -0.013931420973875674, -0.19694396574274..."
1,"[2.0, 0.10145072536268135, 0.0]",0,"[[0.0047779282632206335, 0.0, 0.0], [0.0099028...","[0.0, -0.06780893461152004, -0.372687830109057..."
2,"[2.0, 0.10290145072536269, 0.0]",0,"[[0.004597193584383609, 0.0, 0.0], [0.01080826...","[0.0, -0.05292323279039124, -0.272468358175146..."
3,"[2.0, 0.10435217608804402, 0.0]",0,"[[0.007397813777500801, 0.0, 0.0], [0.01335308...","[0.0, -0.11624923272798071, -0.289476540021439..."
4,"[2.0, 0.10580290145072536, 0.0]",0,"[[0.0062299988834332715, 0.0, 0.0], [0.0081049...","[0.0, 0.16841321371747142, -0.0298498554396644..."


In [7]:
#Data = pd.read_pickle("Sim_Data_Noisy.pkl")
tag_pos = np.vstack(Data['tag_pos'].to_numpy())
antenna_nums = np.vstack(Data['Antenna'].to_numpy())
print(tag_pos.shape)

(2000, 3)


Έπειτα δημιουργούμε έναν πίνακα (tensor) διαστάσεων $(1000, 1000, 4)$ που περιέχει για τα 1000 tags που δημιουργήθηκαν από το simulation, τα 1000 time samples από το μήκος των μετρήσεων, και τα 4 χαρακτηριστικά που μας χρειάζονται , [x_robot,y_robot,z_robot,phase_unwrapped].

***

### Data Augmentation μέσω segmentation

Ορίζουμε την συνάρτηση παρακάτω μέσω της οποίας μετασχηματίζουμε κάθε μία μέτρηση σε ένα πλήθος x που μπορούμε να διαλέξουμε εμείς με y αριθμό μετρήσεων στο κάθε ένα. Αυτό συμβαίνει καθώς για κάθε ένα tag σπάμε τις μετρήσεις σε x (όχι απαραίτητα μη επικαλυπτόμενες) μετρήσεις μεγέθους y.

In [8]:
def sample_random_segments_fn(data, num_samples= 5, segment_length= 400):
    segments = []
    # Data contains ['Phases', 'robot_pos']
    for i in range(len(data)):
        phases = data['Phases'][i]
        robot_pos = data['robot_pos'][i]
        # Get the length of the phases array
        length = len(phases)
        # Generate random start indices for segments
        start_indices = random.sample(range(length - segment_length), num_samples)
        for start in start_indices:
            end = start + segment_length
            segment_phases = phases[start:end]
            segment_robot_pos = robot_pos[start:end]
            # Append the segment data to the list, but subtract the first x-value from the tag pos and robot pos
            segments.append({
                'Phases': segment_phases,
                'robot_pos': segment_robot_pos - segment_robot_pos[0],
                'tag_pos': [data['tag_pos'][i][0] - segment_robot_pos[0][0], data['tag_pos'][i][1], data['tag_pos'][i][2]]
            })
    return segments

In [9]:
# Sample random segments from all the data
random_segments = sample_random_segments_fn(Data, num_samples= 5, segment_length= 400)
# Convert the list of segments to a DataFrame
random_segments_df = pd.DataFrame(random_segments)
random_segments_df.head()

,Phases,robot_pos,tag_pos
0,"[-9.2957602694798, -9.609572904101327, -9.8965...","[[0.0, 0.0, 0.0], [0.004774677828534579, 0.0, ...","[1.7369415305779177, 0.1, 0.0]"
1,"[-6.1017959732055544, -6.167594555618081, -6.2...","[[0.0, 0.0, 0.0], [-0.0009029686782236201, 0.0...","[1.8274908981487963, 0.1, 0.0]"
2,"[-27.44887389425717, -27.392750109251715, -27....","[[0.0, 0.0, 0.0], [0.0005745920054198717, 0.0,...","[1.2405525357968772, 0.1, 0.0]"
3,"[-5.744895640983813, -5.887920268672602, -5.81...","[[0.0, 0.0, 0.0], [0.0026343373035221107, 0.0,...","[1.8381592821599153, 0.1, 0.0]"
4,"[-62.387336622065604, -62.09964894985859, -62....","[[0.0, 0.0, 0.0], [0.0009948973952895734, 0.0,...","[-0.2667734648732236, 0.1, 0.0]"


***

### Linear Interpolation για να έχουμε ίδιο μέγεθος σε όλα τα samples

Μιας και τα δεδομένα μας περιμένουμε να περιέχουν ασυνέχειες, σκεφτόμαστε ότι μέσω της γραμμικής παρεμβολής θα βεβαιωνόμαστε για το σταθερό μήκος του Input στο μοντέλο. Αυτό υλοποιείται παρακάτω.

In [10]:
#Linear interpolation through the random segments data
def linear_interpolation(data, num_of_samples):
    interpolated_data = []
    for i in range(len(data)):
        phases = data['Phases'][i]
        robot_pos = data['robot_pos'][i]
        tag_pos = data['tag_pos'][i]

        # Create the interpolation function
        f_phases = interp1d(np.arange(len(phases)), phases, kind='linear')
        f_robot_pos = interp1d(np.arange(len(robot_pos)), robot_pos, axis=0, kind='linear')

        # Generate new x values for interpolation
        new_x = np.linspace(0, len(phases) - 1, num_of_samples)

        # Interpolate the phases and robot positions
        new_phases = f_phases(new_x)
        new_robot_pos = f_robot_pos(new_x)

        interpolated_data.append({
            'Phases': new_phases,
            'robot_pos': new_robot_pos,
            'tag_pos': tag_pos,

        })
    return interpolated_data


In [11]:
# Interpolate the random segments data
interpolated_segments = linear_interpolation(random_segments_df, num_of_samples=385)
# Convert the list of interpolated segments to a DataFrame
interpolated_segments_df = pd.DataFrame(interpolated_segments)
# Save the interpolated data to a pickle file
interpolated_segments_df.to_pickle(f"{new_folder_path}/Sim_Data_Interpolated.pkl")

Αποθηκεύουμε και το μικρότερο αρχείο χωρίς τις θέσεις του ρομπότ μιας και αυτές είναι προκαθορισμένες και δεν είναι ανάγκη να αποθηκεύονται.